In [1]:
from transformers import pipeline
import json

# Load the JSON file
with open("reddit_results.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Extract text from JSON (titles and comments)
def extract_text(data):
    text = ""
    for entry in data:
        text += entry["title"] + ". "  # Include title
        text += " ".join(entry["comments"]) + " "  # Include comments
    return text

# Get the extracted text
text_data = extract_text(data)

# Summarize the text (limit input to prevent model overload)
summary = summarizer(text_data[:1024], max_length=200, min_length=50, do_sample=False)

# Print the summary
print("Summary:", summary[0]["summary_text"])


Device set to use cuda:0


Summary: The whole point of all these festival is to undergo a little bit of struggle. The poor people continue to run towards religion and get themselves humiliated. #I come from two Indias. How can people even tolerate this kind of nonsense? #I am a hindu and this is really sad.
